In [ ]:
#| default_exp core

# API
- Implementation of fastcloudinit

In [ ]:
#| export
from fastcore.utils import *
import fastcore.xtras, yaml
from textwrap import dedent

In [ ]:
#| export
def ufw(logging="off", def_incoming="deny", def_outgoing="allow", internal=None, **allows):
    rules = [dict(name=k, rule="allow", port=v) for k,v in allows.items()]
    if internal: rules.append(dict(interface=internal, rule="allow", direction="in"))
    defaults=dict(incoming=def_incoming, outgoing=def_outgoing)
    res = dict(enabled=True, logging=logging, defaults=defaults, rules=rules)
    return dict(ufw=res)

In [ ]:
ufw(internal="enp7s0", OpenSSH=22)

{'ufw': {'enabled': True,
  'logging': 'off',
  'defaults': {'incoming': 'deny', 'outgoing': 'allow'},
  'rules': [{'name': 'OpenSSH', 'rule': 'allow', 'port': 22},
   {'interface': 'enp7s0', 'rule': 'allow', 'direction': 'in'}]}}

In [ ]:
#| export
def user(name, pub_key, groups=None, shell="/bin/bash", sudo=True):
    groups = listify(groups)
    if sudo and 'sudo' not in groups: groups.append('sudo')
    sudo = ["ALL=(ALL) NOPASSWD:ALL"] if sudo else []
    return dict(name=name, groups=groups, shell=shell, sudo=sudo, ssh_authorized_keys=[pub_key])

In [ ]:
user('jph', 'mykey', 'docker')

{'name': 'jph',
 'groups': ['docker', 'sudo'],
 'shell': '/bin/bash',
 'sudo': ['ALL=(ALL) NOPASSWD:ALL'],
 'ssh_authorized_keys': ['mykey']}

In [ ]:
#| export
def source(url, keyid, keyserver):
    return dict(source=f"deb [signed-by=$KEY_FILE] {url} $RELEASE stable", keyid=keyid, keyserver=keyserver)

In [ ]:
dsource = source("https://download.docker.com/linux/ubuntu",
                 keyid="9DC858229FC7DD38854AE2D88D81803C0EBFCD88",
                 keyserver="https://download.docker.com/linux/ubuntu/gpg")
dsource

{'source': 'deb [signed-by=$KEY_FILE] https://download.docker.com/linux/ubuntu $RELEASE stable',
 'keyid': '9DC858229FC7DD38854AE2D88D81803C0EBFCD88',
 'keyserver': 'https://download.docker.com/linux/ubuntu/gpg'}

In [ ]:
#| export
def apt(unattended=False, autoclean=30, email='', auto_reboot=False, **sources):
    unattended = "1" if unattended else "0"
    auto_reboot = "true" if auto_reboot else "false"
    apt_conf = f"""\
APT::Periodic::Update-Package-Lists "1";
APT::Periodic::Download-Upgradeable-Packages "1";
APT::Periodic::AutocleanInterval "7";
APT::Periodic::Unattended-Upgrade "{unattended}";
Unattended-Upgrade::Automatic-Reboot "{auto_reboot}";
"""
    if email: apt_conf += 'Unattended-Upgrade::Mail "{email}";\n'
    res = dict(conf=apt_conf)
    if sources: res['sources'] = sources
    return dict(apt=res)

In [ ]:
apt(docker=dsource)

{'apt': {'conf': 'APT::Periodic::Update-Package-Lists "1";\nAPT::Periodic::Download-Upgradeable-Packages "1";\nAPT::Periodic::AutocleanInterval "7";\nAPT::Periodic::Unattended-Upgrade "0";\nUnattended-Upgrade::Automatic-Reboot "false";\n',
  'sources': {'docker': {'source': 'deb [signed-by=$KEY_FILE] https://download.docker.com/linux/ubuntu $RELEASE stable',
    'keyid': '9DC858229FC7DD38854AE2D88D81803C0EBFCD88',
    'keyserver': 'https://download.docker.com/linux/ubuntu/gpg'}}}}

In [ ]:
#| export
def systemd(items):
    units = [dict(name=k, dropin=True, content=v) for k,v in items.items()]
    return dict(systemd=dict(units=units))

In [ ]:
systemd({"systemd-journald.service":"[Journal]\nSystemMaxUse=50M\n"})

{'systemd': {'units': [{'name': 'systemd-journald.service',
    'dropin': True,
    'content': '[Journal]\nSystemMaxUse=50M\n'}]}}

In [ ]:
#| export
def log_rotate(freq="weekly", num_keep=7, fname="00-cloud-init-global"):
    snippet = f"""\
/var/log/*.log {{
    {freq}
    rotate {num_keep}
    compress
    su root adm
    create
    missingok
}}
"""
    return dict(path=f"/etc/logrotate.d/{fname}", owner="root:root", permissions="0644", content=snippet)

In [ ]:
#| export
def phone_home(url):
    if not url: return {}
    res = dict(url=url, post=["instance_id", "hostname"], tries=5)
    return dict(phone_home=res)

In [ ]:
#| export
def reboot(enable=True, message="Rebooting"):
    if not enable: return {}
    return dict(power_state=dict( mode="reboot", message=message, timeout=1, condition=True))

In [ ]:
#| export
def mounts(devices):
    if not devices: return {}
    return dict(mounts=devices)

In [ ]:
#| export
def cloud_init_config(hostname, username, ssh_pub_key, email='',
                      devices=None, ping_host=None, packages=None, **sources):
    cfg = dict(
        hostname=hostname, preserve_hostname=False,
        users=[user(username, ssh_pub_key, groups="docker")],
        packages=listify(packages), package_update=True, package_upgrade=True,
        disable_root=True, ssh_pwauth=False,
        **ufw(internal="enp7s0", OpenSSH=22),
        **apt(**sources),
        **systemd({"systemd-journald.service":"[Journal]\nSystemMaxUse=250M\n"}),
        write_files=[ log_rotate() ],
        **mounts(devices),
        **phone_home(ping_host),
        **reboot(),
    )
    return "#cloud-config\n" + yaml.safe_dump(cfg, sort_keys=False, width=1_000_000)

In [ ]:
packages = [ "unattended-upgrades", "vim-nox", "python3", "rsync", "ubuntu-drivers-common", "python3-pip", "ack", "lsyncd", "wget", "bzip2", "ca-certificates", "git", "build-essential", "software-properties-common", "curl", "grep", "sed", "dpkg", "tmux", "less", "htop", "openssh-client", "python-is-python3", "python3-dev", "cron", "gnupg", "docker-ce", "docker-ce-cli", "containerd.io", "docker-buildx-plugin", "docker-compose-plugin" ]

dsource = source(
    "https://download.docker.com/linux/ubuntu", 
    keyid="9DC858229FC7DD38854AE2D88D81803C0EBFCD88", 
    keyserver="https://download.docker.com/linux/ubuntu/gpg")
devices = [ ['mydevice', "/data", "ext4", "defaults,nofail", "0", "0"] ]

res = cloud_init_config('myhost', 'jph', 'mykey', 'j@answer.ai',
                        devices=devices, ping_host='https://ping.me', packages=packages, docker=dsource)
print(res)

#cloud-config
hostname: myhost
preserve_hostname: false
users:
- name: jph
  groups:
  - docker
  - sudo
  shell: /bin/bash
  sudo:
  - ALL=(ALL) NOPASSWD:ALL
  ssh_authorized_keys:
  - mykey
packages:
- unattended-upgrades
- vim-nox
- python3
- rsync
- ubuntu-drivers-common
- python3-pip
- ack
- lsyncd
- wget
- bzip2
- ca-certificates
- git
- build-essential
- software-properties-common
- curl
- grep
- sed
- dpkg
- tmux
- less
- htop
- openssh-client
- python-is-python3
- python3-dev
- cron
- gnupg
- docker-ce
- docker-ce-cli
- containerd.io
- docker-buildx-plugin
- docker-compose-plugin
package_update: true
package_upgrade: true
disable_root: true
ssh_pwauth: false
ufw:
  enabled: true
  logging: 'off'
  defaults:
    incoming: deny
    outgoing: allow
  rules:
  - name: OpenSSH
    rule: allow
    port: 22
  - interface: enp7s0
    rule: allow
    direction: in
apt:
  conf: 'APT::Periodic::Update-Package-Lists "1";

    APT::Periodic::Download-Upgradeable-Packages "1";

    APT::Per

## export -

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()